Where to find your access token :https://blogs.dropbox.com/developers/2014/05/generate-an-access-token-for-your-own-account/

In [1]:
import dropboxdrivefs as dbx
import logging
import os

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

In [2]:
token = "*****"
fs = dbx.DropboxDriveFileSystem(token=token)

## Create a folder

In [3]:
fs.mkdir("/Data/test_dropbox")

INFO:Request to files/create_folder_v2
INFO:The directory/Data/test_dropbox has  been created.


In [4]:
fs.info("/Data/test_dropbox")

INFO:Request to files/get_metadata


{'id': 'id:pgKT2sXX3sMAAAAAAAAJDg',
 'shared_folder_id': None,
 'sharing_info': None,
 'property_groups': None,
 'name': '/Data/test_dropbox',
 'size': None,
 'type': 'directory'}

## Upload

In [5]:
# Upload a file

fs.put_file("example_file/test1.txt", "/Data/test_dropbox/text1.txt")

INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish


In [6]:
# Upload a folder with two files 

fs.upload("example_file/test_db", "/Data/test_dropbox/", recursive=True)

INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish
INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish


## Download a file

In [7]:
fs.download("/Data/test_dropbox/text1.txt", "example_file/test.txt")

INFO:Request to files/get_metadata
INFO:Request to files/get_metadata
INFO:Request to files/get_temporary_link
INFO:Request to files/get_metadata


## Write in a file

In [8]:
with fs.open("/Data/test_dropbox/test.txt", mode='w') as f:
    f.write('Hello')

INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish


Be careful, the mode "append" does not append the existing file. It creates a new file with the text wrotten by adding (2) in the path name. If this new file is identical with the existing file, nothing is uploaded. 

In [9]:
with fs.open("/Data/test_dropbox/test.txt", mode='a') as f:
    f.write('world')

INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish


## Read a file 

In [10]:
with fs.open("/Data/test_dropbox/test.txt", mode="r") as f:
    print(f.read())

INFO:Request to files/get_temporary_link
INFO:Request to files/get_metadata


Hello


## Read with intake

Note that the path used has three ///: The path needs to be dropbox:// + /folder1/folder2 

Without it, the dropbox sdk used in background will take as cloud path : /folder2 instead of /folder1/folder2

In [11]:
import intake

In [12]:
file_intake = intake.open_textfiles("dropbox:///Data/test_dropbox/test.txt",storage_options={"token":token})


In [13]:
file_intake.read()

['Hello']

## List all files before removing the folder online

In [14]:
items = fs.ls("/Data/test_dropbox")
for item in items:
    print(item)

print("\nAvailable info for folder type:")
print(list(items[0].keys()))

print("\nAvailable info for file type:")
print(list(items[-1].keys()))

INFO:Request to files/list_folder


{'id': 'id:pgKT2sXX3sMAAAAAAAAJEQ', 'shared_folder_id': None, 'sharing_info': None, 'property_groups': None, 'name': '/Data/test_dropbox/test_db', 'size': None, 'type': 'directory'}
{'id': 'id:pgKT2sXX3sMAAAAAAAAJDw', 'client_modified': datetime.datetime(2024, 1, 18, 18, 54, 39), 'server_modified': datetime.datetime(2024, 1, 18, 18, 54, 39), 'rev': '60f3ce546ce3e0f9d7ed4', 'size': 5, 'media_info': None, 'symlink_info': None, 'sharing_info': None, 'is_downloadable': True, 'export_info': None, 'property_groups': None, 'has_explicit_shared_members': None, 'content_hash': '9595c9df90075148eb06860365df33584b75bff782a510c6cd4883a419833d50', 'file_lock_info': None, 'name': '/Data/test_dropbox/text1.txt', 'type': 'file'}
{'id': 'id:pgKT2sXX3sMAAAAAAAAJEw', 'client_modified': datetime.datetime(2024, 1, 18, 18, 54, 44), 'server_modified': datetime.datetime(2024, 1, 18, 18, 54, 45), 'rev': '60f3ce5a244e00f9d7ed4', 'size': 5, 'media_info': None, 'symlink_info': None, 'sharing_info': None, 'is_down

In [15]:
items = fs.ls("/Data/test_dropbox", detail=False)
for item in items:
    print(item)

INFO:Request to files/list_folder


/Data/test_dropbox/test_db
/Data/test_dropbox/text1.txt
/Data/test_dropbox/test.txt
/Data/test_dropbox/test (1).txt


In [16]:
items = fs.ls("/Data/test_dropbox", detail=False, recursive=True)
for item in items:
    print(item)

INFO:Request to files/list_folder


/Data/test_dropbox
/Data/test_dropbox/test_db
/Data/test_dropbox/text1.txt
/Data/test_dropbox/test_db/test1.txt
/Data/test_dropbox/test_db/test2.txt
/Data/test_dropbox/test.txt
/Data/test_dropbox/test (1).txt


In [17]:
fs.rm("/Data/test_dropbox")
os.remove("example_file/test.txt")

INFO:Request to files/delete_v2
INFO:The directory/Data/test_dropbox has been erased.
